In [2]:
import fridge
import fridge.utilities.h5Interface as h5i
import fridge.utilities.reactorInterface as RI
import os
import pickle

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import bb_sfr_opt
import controller
import time

# SFR Optimization

This example seeks to optimize a SFR design utilizing the blackboard framework.
We start first by taking a look at the SFR design data using the `h5` add `reactor` interfaces built into `FRDIGe` [cite].
For this we'll examine the objectives that will drive the optimization process: cycle length, plutonium mass, reactivity swing, and core average burnup.
Upon examination, we note that we have removed a significant portion of the design space by ensuring that the reactor design has a cycle length greater than 0 days.

In [6]:
h5_interface = h5i.h5Interface()
h5_interface.read_h5('/Users/ryanstewart/projects/sfr_database/SFR_DB.h5')

In [41]:
cycle_length_list = []
rx_swing_list = []
height = []
smear = []
pu_content = []
bu = []
pu_mass = []
print(len(h5_interface.h5file.items()))
for rx, core in h5_interface.h5file.items():
    try:
        rx_ = RI.reactorInterface(core)
        cycle_length = rx_.extrapolate_value('keff', 'time', 1.0)
        rx_swing = rx_.get_reactivity_swing(1.0, cycle_length)
        rx_swing_per_day = rx_swing/cycle_length * 30
        avg_bu = rx_.get_assembly_avg(cycle_length, 'burnup')
        avg_bu_per_day = avg_bu/cycle_length * 100
        pu = rx_.get_assembly_pu_mass() * 78 * 365 / (3 * cycle_length)
        if cycle_length > 0:
            height.append(rx_.rx['independent variables']['height'][0])
            smear.append(rx_.rx['independent variables']['smear'][0])
            pu_content.append(rx_.rx['independent variables']['pu_content'][0])
            cycle_length_list.append(cycle_length)
            rx_swing_list.append(rx_swing_per_day)
            bu.append(avg_bu)
            pu_mass.append(pu)
    except (KeyError, TypeError):
        print('didnt get {}'.format(core))

78


In [42]:
for s, h, p, pu, c, b, r in zip(smear, height, pu_content, pu_mass, cycle_length_list, bu, rx_swing_list):
    print(s, h, round(p,2), round(pu,0), round(c,0), round(b,0), round(r,0))

50.0 60.0 1.0 8228.0 60.0 12.0 1075.0
50.0 70.0 0.75 5293.0 81.0 14.0 875.0
50.0 70.0 1.0 2709.0 212.0 37.0 902.0
50.0 80.0 0.56 3678.0 98.0 15.0 743.0
50.0 80.0 0.75 2188.0 224.0 35.0 763.0
50.0 80.0 1.0 1791.0 367.0 57.0 782.0
55.0 55.0 0.88 6707.0 65.0 13.0 1053.0
55.0 65.0 0.88 2133.0 240.0 42.0 866.0
55.0 75.0 0.4 2650.0 101.0 15.0 706.0
55.0 75.0 0.88 1399.0 423.0 63.0 738.0
60.0 50.0 0.75 8039.0 46.0 9.0 1050.0
60.0 50.0 1.0 2945.0 167.0 34.0 1062.0
60.0 60.0 0.56 2626.0 124.0 21.0 826.0
60.0 60.0 0.75 1812.0 243.0 42.0 852.0
60.0 60.0 1.0 1565.0 378.0 65.0 868.0
60.0 70.0 0.56 1204.0 315.0 47.0 703.0
60.0 70.0 0.75 1162.0 443.0 65.0 724.0
60.0 70.0 1.0 1168.0 591.0 87.0 739.0
60.0 70.0 0.25 2451.0 69.0 10.0 675.0
60.0 80.0 0.56 842.0 515.0 67.0 602.0
60.0 80.0 0.75 891.0 660.0 85.0 614.0
60.0 80.0 1.0 959.0 823.0 106.0 629.0
60.0 80.0 0.25 784.0 247.0 32.0 573.0
65.0 55.0 0.4 3794.0 61.0 11.0 804.0
65.0 55.0 0.88 1434.0 357.0 62.0 858.0
65.0 65.0 0.4 1008.0 271.0 40.0 676.0
65.

# Starting the Blackboard System

In [1]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_sfr_opt.BbSfrOpt)
#bb.set_attr(objective_ranges={'cycle length': (50, 750), 
#                             'reactivity swing': (0, 750), 
#                             'burnup': (0, 16), 
#                             'pu mass': (0, 1000)})
bb.set_attr(objective_ranges={'reactivity swing': (0, 750), 
                             'burnup': (0, 200),})
bb.add_abstract_lvl(3, {'reactor parameters': {'height': float, 
                                               'smear': float, 
                                               'pu_content': float, 
                                               'reactivity swing': float, 
                                               'burnup': float,}})
bb.add_panel(3, ['new','old'])
bb.set_attr(objectives=['reactivity swing', 'burnup'])
bb.set_attr(objective_goals={'reactivity swing': 'lt',
                              'burnup':'gt'})

#bb.set_attr(objective_ranges={'keff': (1.0, 1.3), 
#                             'doppler': (-1.5, 0), 
#                             'void': (-250, 0), 
#                             'pu_content': (0, 1.0)})
#bb.add_abstract_lvl(3, {'reactor parameters': {'height': float, 
#                                               'smear': float, 
#                                               'pu_content': float, 
#                                               'keff': float, 
#                                               'doppler': float, 
#                                               'void': float}})
#bb.add_panel(3, ['new','old'])

#bb.set_attr(objectives=['keff', 'doppler', 'void', 'pu_content'])
#bb.set_attr(objective_goals={'keff': 'gt',
#                              'void':'lt',
#                              'doppler': 'lt',
#                              'pu_content': 'lt'})
#bb.set_attr(problem='prelims')

model = 'lr'
with open('/Users/ryanstewart/projects/Dakota_Interface/GA_BB/sm_{}.pkl'.format(model), 'rb') as pickle_file:
    sm_ga = pickle.load(pickle_file)
bb.set_attr(sm_type=model)
bb.set_attr(_sm=sm_ga)
#bb.set_attr(_DEBUG=True)
#bb.generate_sm()
bb.set_attr(required_agents=[karp.KaRpExplore,karp.KaRpExploit,kabr.KaBr_lvl2,kabr.KaBr_lvl3])
bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_1')
bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_2')
bb.connect_agent(karp.KaRpExploit, 'ka_rp_exploit')
bb.connect_agent(kabr.KaBr_lvl2, 'ka_br_lvl2')
bb.connect_agent(kabr.KaBr_lvl3, 'ka_br_lvl3')
#ka = bb.get_attr('_proxy_server')
#ka1 = ka.proxy('ka_rp_explore_1')
#ka1.set_attr(_DEBUG=True)
#ka1 = ka.proxy('ka_rp_explore_2')
#ka1.set_attr(_DEBUG=True)

#bb.connect_agent(kabr.KaBr_lvl3, 'ka_br_lvl3_2')
#sm = bb.get_attr('_sm')
#mse_score = sm.models[model]['mse_score']
#print('MSE Score: {}'.format(mse_score))

SyntaxError: invalid syntax (<ipython-input-1-33ea7c1c8e8e>, line 11)

In [5]:
models = ['ann', 'lr', 'pr', 'mars' , 'gpr', 'rf']
for model in models:
    bb.set_attr(sm_type=model)
    bb.generate_sm()
    sm = bb.get_attr('_sm')
    mse_score = sm.models[model]['mse_score']
    print(mse_score)

0.24271961819367802
0.08937285282748705
0.19365672680083224
0.1294654243241577
0.11844519218721544
0.08170566765995858


In [5]:
#while not bb.get_attr('_complete'):
#    i=0
#bb.connect_agent(kabr.KaBr_lvl3, 'ka_br_lvl3_2')
bb.set_attr(total_solutions=50)
for i in range(250):

    bb.publish_trigger()
    trig_num = bb.get_attr('_trigger_event')
    responses = False
    while not responses:
        time.sleep(0.1)
        if len(bb.get_attr('_kaar')[trig_num]) == 5:
            responses = True
    bb.controller()
    bb.set_attr(_new_entry=False)
    bb.send_executor()
    time_sleep = 0
#    bb.log_info((bb.get_attr('_new_entry'), i))
    while bb.get_attr('_new_entry') == False:
        print('waiting {} (s) for {} i={}'.format(round(time_sleep,2), bb.get_attr('_ka_to_execute'), i))
        time.sleep(0.1)
        time_sleep += 0.1
        if time_sleep > 200:
            break
    if len(bb.get_attr('_kaar')) % 25 == 0:
        bb.write_to_h5()
        bb.plot_progress()
        bb.diagnostics_replace_agent()
        lvls = bb.get_attr('abstract_lvls')

        try:
            print('')
            print('level 1 new: {}, old: {}, frac: {}'.format(len(lvls['level 1']['new']),len(lvls['level 1']['old']),len(lvls['level 1']['new'])/len(lvls['level 1']['old'])))
            print('level 2 new: {}, old: {}, frac: {}'.format(len(lvls['level 2']['new']),len(lvls['level 2']['old']),len(lvls['level 2']['new'])/len(lvls['level 2']['old'])))
            print('level 3 new: {}, old: {}, frac: {}'.format(len(lvls['level 3']['new']),len(lvls['level 3']['old']),len(lvls['level 3']['new'])/len(lvls['level 3']['old'])))
            print('')
        except:
            pass
    if bb.get_attr('_complete'):
        break

waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=0
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=1
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=2
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=3
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=4
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=5
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=6
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=7
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=8
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=9
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=10
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=11
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=12
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=13
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=14
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=15
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=16
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=17
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=18
waiting 0 (s) for ('ka_rp_explore_2', 0.5) i=19
waiting 0 (s) for ('ka_rp_explore_1', 0.5) i=20
wa

KeyboardInterrupt: 

In [7]:
br_lvl2 = 0
br_lvl3 = 0
rp_pert = 0
rp_mc = 0
for agents in bb.get_attr('_kaar').values():
    val = 0
    val_max = 0
    for a, val in agents.items():
        if val > val_max:
            agent = a
            val_max = val
    if 'ka_br_lvl2' in agent:
        br_lvl2 += 1
    if 'ka_br_lvl3' in agent:
        br_lvl3 += 1
    if 'ka_rp_exploit' in agent:
        rp_pert += 1
    if 'ka_rp_explore' in agent:
        rp_mc += 1
        
print(br_lvl2, br_lvl3, rp_pert, rp_mc)

304 377 224 345


In [ ]:
kas = {'ka_rp_explore': ka_rp.KaRpExplore, 
       'ka_rp_exploit': ka_rp.KaRpExploit,
       'ka_br_lvl3': ka_br.KaBr_lvl3,
       'ka_br_lvl2': ka_br.KaBr_lvl2}
bb_controller = controller.Controller(bb_name='bb', bb_type=bb_sfr.BbSfrOpt, ka=kas, archive='sfr_opt', agent_wait_time=20)
bb_controller.bb.set_attr(total_solutions=50)
    
bb_controller.run_single_agent_bb()
